In [2]:
import pandas as pd

df = pd.read_csv('train.tsv', sep='\t')

In [9]:
from sklearn.feature_extraction.text import CountVectorizer

df = df[:1000]
vectorizer = CountVectorizer(binary=True)
text_data = df["Phrase"]
label_data = df["Sentiment"]

text_data = vectorizer.fit_transform(text_data)

0      1
1      2
2      2
3      2
4      2
5      2
6      2
7      2
8      2
9      2
10     2
11     2
12     2
13     2
14     2
15     2
16     2
17     2
18     2
19     2
20     2
21     3
22     3
23     2
24     2
25     2
26     2
27     2
28     2
29     2
      ..
970    1
971    2
972    1
973    2
974    2
975    1
976    1
977    2
978    1
979    2
980    1
981    2
982    3
983    1
984    2
985    2
986    2
987    2
988    1
989    1
990    1
991    1
992    1
993    2
994    2
995    3
996    2
997    2
998    1
999    1
Name: Sentiment, Length: 1000, dtype: int64

In [11]:
import torch
from torchtext import data

t = torch.from_numpy(text_data.todense())

#ajout du cas où on ne connait pas le mot
voca = vectorizer.get_feature_names() + list(['<unk>'])
unk = torch.zeros([t.shape[0],1], dtype=torch.int64)
t = torch.cat((t, unk), 1)

#train_data, test_data = datasets.IMDB.splits(t, label_data)

ModuleNotFoundError: No module named 'torchtext'

In [ ]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator

In [7]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.rnn = nn.RNN(embedding_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x):

        #x = [sent len, batch size]
        
        embedded = self.embedding(x)
        
        #embedded = [sent len, batch size, emb dim]
        
        output, hidden = self.rnn(embedded)
        
        #output = [sent len, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        
        assert torch.equal(output[-1,:,:], hidden.squeeze(0))
        
        return self.fc(hidden.squeeze(0))

In [8]:
input_dim = len(voca)
embedding_dim = 100
hidden_dim = 250
output_dim = 1

model = RNN(input_dim, embedding_dim, hidden_dim, output_dim)